In [455]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import warnings
import json
warnings.filterwarnings('ignore')

In [456]:
tweet_df = pd.read_csv('Datasets/data.csv',encoding="ISO-8859-1")
t_df = pd.read_csv('Datasets/data1.csv',encoding="ISO-8859-1")
tweet_df.drop(tweet_df.columns.difference(['created_at','full_text']),axis=1, inplace=True)
t_df.drop(t_df.columns.difference(['created_at','tweet']),axis=1, inplace=True)
tweet_df = tweet_df.rename(columns={'full_text': 'tweet'})
frames=[tweet_df, t_df]
final_df = pd.concat(frames, ignore_index=True)
final_df

,created_at,tweet
0,2022-07-13T14:29:17.000Z,PNP OIC PLTGEN Danao Jr. confirmed that two me...
1,2022-07-13T12:10:52.000Z,This anti-illegal drug operation also led to t...
2,2022-07-13T11:20:38.000Z,A regional level most wanted person for illega...
3,2022-07-12T13:18:14.000Z,Three former members of a CTG in Masbate are n...
4,2022-07-12T11:53:06.000Z,A 37-year-old drug suspect was busted in an an...
...,...,...
1485,2021-09-02 12:12:11 Malay Peninsula Standard Time,ELEAZAR: 92% OF PNP PERSONNEL RECEIVED VACCINE...
1486,2021-09-01 14:35:45 Malay Peninsula Standard Time,ELEAZAR: POLICEMEN CHARGED WITH MURDER BY DOJ ...
1487,2021-09-01 12:41:19 Malay Peninsula Standard Time,POLICE CONDUCT MOBILE LIBRARY FOR CHILDREN IN ...
1488,2021-09-01 12:09:47 Malay Peninsula Standard Time,IAS RECOMMENDATION ON POLICEMAN INVOLVED IN FA...


In [457]:
final_df = pd.read_csv('Datasets/type_data.csv',encoding="ISO-8859-1")

In [458]:
import re
from nltk.tokenize import word_tokenize
tagalog_stops = pd.read_csv("Datasets/tagalog_stopwords.csv").a.to_list()
stop_words = stopwords.words('english') + tagalog_stops
def clean_tweets(text):
    text = text.lower()
    text = re.sub(r'RT @[\w]*','',str(text))
    text = re.sub(r'@[\w]*','',str(text))
    text = re.sub(r'#([a-zA-Z0-9_]{1,50})','',str(text))
    text = re.sub(r'https?://[A-Za-z0-9./]*','',str(text))
    text = re.sub(r'\n','',str(text))
    text = re.sub(r'[^\x00-\x7F\x80-\xFF\u0100-\u017F\u0180-\u024F\u1E00-\u1EFF]', u'', str(text))
    text = re.sub(r'[^\x00-\x7f]', '', str(text))
    text = re.sub(r'[^\w\s]', '', str(text))
    text_tokens = word_tokenize(text)
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)
final_df['tweet'] = final_df['tweet'].apply(lambda x: clean_tweets(x))
final_df

,created_at,tweet,type
0,2022-07-13T14:29:17.000Z,pnp oic pltgen danao jr confirmed two members ...,terrorism
1,2022-07-13T12:10:52.000Z,antiillegal drug operation also led seizure 25...,drug
2,2022-07-13T11:20:38.000Z,regional level wanted person illegal drugs nab...,drug
3,2022-07-12T13:18:14.000Z,three former members ctg masbate reintegrated ...,terrorism
4,2022-07-12T11:53:06.000Z,37yearold drug suspect busted antiillegal drug...,drug
...,...,...,...
335,2021-09-07 11:45:49 Malay Peninsula Standard Time,2 arrested manufacture selling swab test resul...,fraud
336,2021-09-06 15:46:01 Malay Peninsula Standard Time,ias findings death minor laguna antidrug opera...,NaN
337,2021-09-06 11:49:04 Malay Peninsula Standard Time,eleazar hails pnpafp operation intercepted amm...,terrorism
338,2021-09-02 15:22:16 Malay Peninsula Standard Time,eleazar hails arrest gun syndicate leader coho...,terrorism


In [459]:
json_city = json.load(open("Datasets/cities.geojson", "r"))
id_map = {}
city_map = {}
province_map = {}
region_map = {}
index = 0
for feature in json_city["features"]:
    #feature["id"] = feature["properties"]["ID_2"]
    feature["id"] = index
    if feature["geometry"] != None:
        id_map[feature["properties"]["NAME_1"].lower() + " " + feature["properties"]["NAME_2"].lower()] = feature["id"]
        words = feature["properties"]["NAME_2"].lower().split()
        for w in words:
            city_map[w] = feature["id"]
        words = feature["properties"]["NAME_1"].lower().split()
        for w in words:
            province_map[w] = feature["id"]
        region_map[feature["properties"]["REGION"].lower().split("(")[0].strip()] = feature["id"]
    index += 1

In [460]:
from nltk import ngrams
final_df['coords'] = np.NaN
no_coords = final_df[['tweet','coords','type']]

def locate_tweet(text):
    found = False
    n_gram = 1
    while not found:
        # spliting the sentence
        words = ngrams(text.split(), n_gram)
        result = ""
        text_tokens = word_tokenize(text)
        city = ""
        province = ""
        init_loc = ""
        for w in words:
            w = re.sub(r'[^a-zA-Z]', '', str(w))
            if city_map.__contains__(w) and city == "":
                city = w
                init_loc = w + " " + init_loc
            if province_map.__contains__(w) and province == "":
                province = w
                init_loc = init_loc  + w
            if city != "" and province != "":
                init_loc.strip()
                break;
        index_loc = -1
        if id_map.__contains__(init_loc):
            index_loc = id_map[init_loc]
        elif province != "":
            index_loc = province_map[province]
        elif city != "":
            index_loc = city_map[city]
        else:
            if n_gram == 3:
                return np.NaN
            n_gram += 1
        if index_loc == -1 and n_gram == 3:
            return np.NaN
        sz = len(json_city["features"][int(index_loc)]["geometry"]["coordinates"])
        sz2 = json_city["features"][int(index_loc)]["geometry"]["coordinates"][np.random.randint(sz)]
        loc = sz2[np.random.randint(len(sz2))]
        region = json_city["features"][int(index_loc)]["properties"]["REGION"]
        location = re.findall('\\d+.\\d+', str(loc))
        if loc is not None:
            found = True
    return str(float(location[1])+-np.random.randint(2))+"/"+str(float(location[0])+0.25)+":"+init_loc+":"+region

no_coords['coords'] = no_coords['tweet'].apply(locate_tweet)

regions = json.load(open("Datasets/Regions.json", "r"))

# Add Names
def add_location_name(text):
    return text.split(":")[1]
def add_region(text):
    return text.split(":")[2]

no_coords['loc_name'] = no_coords['coords'].apply(add_location_name)
no_coords['region_name'] = no_coords['coords'].apply(add_region)
#no_coords['region_id'] = no_coords['region_name'].apply(add_region_id)
no_coords = no_coords.loc[no_coords['loc_name'] != ""]
# Preprocess
def preprocess_coords(coords):
    numbers = re.findall('\\d+', coords)
    return numbers[2]+"."+numbers[3]+"/"+numbers[0]+"."+numbers[1]

no_coords['coords'] = no_coords['coords'].apply(preprocess_coords)

no_coords

,tweet,coords,type,loc_name,region_name
0,pnp oic pltgen danao jr confirmed two members ...,124.288208/13.04128,terrorism,danao sorsogon,Bicol Region (Region V)
3,three former members ctg masbate reintegrated ...,124.040558/11.19639,terrorism,masbate masbate,Bicol Region (Region V)
4,37yearold drug suspect busted antiillegal drug...,123.679863/7.61541,drug,san,Zamboanga Peninsula (Region IX)
7,police arrested monday member communist terror...,122.009827/17.57057,terrorism,cagayan cagayan,Cagayan Valley (Region II)
9,municipallevel wanted person bohol finally arr...,124.50071/9.62657,wanted,bohol,Central Visayas (Region VII)
...,...,...,...,...,...
334,p34b worth shabu seized 4 bigtime drug traffic...,120.427498/13.74056,drug,zambales,Central Luzon (Region III)
335,2 arrested manufacture selling swab test resul...,121.241058/14.732831,fraud,manila manila,Metropolitan Manila
336,ias findings death minor laguna antidrug opera...,121.601479/13.18381,NaN,laguna laguna,CALABARZON (Region IV-A)
337,eleazar hails pnpafp operation intercepted amm...,126.817047/7.34229,terrorism,caraga,Davao Region (Region XI)


In [461]:
no_coords.reset_index(inplace = True)
specific_df = no_coords.loc[no_coords['type'] != ""].reset_index()[['coords','type']]
specific_df

,coords,type
0,124.288208/13.04128,terrorism
1,124.040558/11.19639,terrorism
2,123.679863/7.61541,drug
3,122.009827/17.57057,terrorism
4,124.50071/9.62657,wanted
...,...,...
199,120.427498/13.74056,drug
200,121.241058/14.732831,fraud
201,121.601479/13.18381,NaN
202,126.817047/7.34229,terrorism


In [462]:
# Heat map test
import folium
from folium.plugins import HeatMap

heat_list = []
for _loc in range(specific_df.shape[0]):
#for _loc in range(10):
    temp = specific_df.coords[_loc].split("/")
    heat_list.append([float(temp[1]), float(temp[0]), 1])
#heat_list = [[11.33877, 124.552177, 0.8]]

mapObj = folium.Map([12, 122], zoom_start=6)
gradient={'0.0':'Navy', '0.25':'Blue','0.5':'Green', '0.75':'Yellow','1': 'Red'}
#HeatMap(data=heat_list, gradient=gradient, radius=25, blur = 10, min_opacity = 0.25, max_val = 0.0005).add_to(mapObj)
HeatMap(heat_list,radius=25, max_zoom=13).add_to(mapObj)
mapObj.save("output.html")

In [463]:
def random_test(text):
    return np.random.randint(100)
no_coords["dummy"] = no_coords["type"].apply(random_test)
no_coords

,index,tweet,coords,type,loc_name,region_name,dummy
0,0,pnp oic pltgen danao jr confirmed two members ...,124.288208/13.04128,terrorism,danao sorsogon,Bicol Region (Region V),72
1,3,three former members ctg masbate reintegrated ...,124.040558/11.19639,terrorism,masbate masbate,Bicol Region (Region V),59
2,4,37yearold drug suspect busted antiillegal drug...,123.679863/7.61541,drug,san,Zamboanga Peninsula (Region IX),79
3,7,police arrested monday member communist terror...,122.009827/17.57057,terrorism,cagayan cagayan,Cagayan Valley (Region II),27
4,9,municipallevel wanted person bohol finally arr...,124.50071/9.62657,wanted,bohol,Central Visayas (Region VII),75
...,...,...,...,...,...,...,...
199,334,p34b worth shabu seized 4 bigtime drug traffic...,120.427498/13.74056,drug,zambales,Central Luzon (Region III),98
200,335,2 arrested manufacture selling swab test resul...,121.241058/14.732831,fraud,manila manila,Metropolitan Manila,80
201,336,ias findings death minor laguna antidrug opera...,121.601479/13.18381,NaN,laguna laguna,CALABARZON (Region IV-A),57
202,337,eleazar hails pnpafp operation intercepted amm...,126.817047/7.34229,terrorism,caraga,Davao Region (Region XI),52


In [467]:
import plotly.express as px

fig = px.choropleth_mapbox(
    no_coords,
    locations="region_name",
    geojson=regions,
    color="dummy",
    featureidkey="properties.REGION",
    hover_name="region_name",
    hover_data=["type"],
    mapbox_style="carto-positron",
    center={"lat": 12, "lon": 122},
    zoom=5,
)
fig.write_html("file.html")